In [1]:
pip install fitz

  Using cached fitz-0.0.1.dev2-py2.py3-none-any.whl.metadata (816 bytes)
  Using cached configobj-5.0.9-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached configparser-7.2.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached nibabel-5.3.2-py3-none-any.whl.metadata (9.1 kB)
  Using cached nipype-1.10.0-py3-none-any.whl.metadata (7.1 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 165.2 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.8 kB 187.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 219.4 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 239.5 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 232.1 kB/s eta 0:00:00
  Using cached pandas-2.2.3-cp312-cp312-win_amd64

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install PyMuPDF

   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.6 MB 217.9 kB/s eta 0:01:16
   ---------------------------------------- 0.1/16.6 MB 328.6 kB/s eta 0:00:51
   ---------------------------------------- 0.1/16.6 MB 467.6 kB/s eta 0:00:36
   ---------------------------------------- 0.2/16.6 MB 615.9 kB/s eta 0:00:27
    --------------------------------------- 0.2/16.6 MB 689.2 kB/s eta 0:00:24
    --------------------------------------- 0.2/16.6 MB 600.7 kB/s eta 0:00:28
    --------------------------------------- 0.4/16.6 MB 849.3 kB/s eta 0:00:20
   - -------------------------------------- 0.5/16.6 MB 1.0 MB/s eta 0:00:16
   - -------------------------------------- 0.6/16.6 MB 1.1 MB/s eta 0:00:16
   - --------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pymupdf pandas tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 131.3 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.7 kB 186.2 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 216.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/78.5 kB ? eta -:--:--
   ------------------------------- -------- 61.4/78.5 kB 656.4 kB/s eta 0:00:01
   ------------------------------------ --- 71.7/78.5 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 78.5/78.5 kB 484.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import pandas as pd
import re
import fitz  # PyMuPDF
from tqdm import tqdm

In [5]:
def extract_metadata(text):
    """Извлекает метаданные из текста PDF"""
    metadata = {
        "title": "",
        "author": "",
        "tags": [],
        "hubs": []
    }
    
    # Извлечение заголовка (первая строка после некоторых пустых строк)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if lines:
        metadata["title"] = lines[0]
    
    # Поиск автора (формат "Автор · дата")
    author_pattern = r"([A-Za-zА-Яа-яЁё0-9_]+)\s+[\d]+\s+[а-я]+\s+в\s+[\d:]+"
    match = re.search(author_pattern, text)
    if match:
        metadata["author"] = match.group(1)
    
    # Извлечение тегов (строка, начинающееся с "Теги:" или "Тети:")
    tags_pattern = r"(?:Теги|Тети):\s*([^\n]+)"
    tags_match = re.search(tags_pattern, text)
    if tags_match:
        metadata["tags"] = [tag.strip() for tag in tags_match.group(1).split(',')]
    
    # Извлечение хабов (ищем строку, начинающуюся с "Хабы:")
    hubs_pattern = r"Хабы:\s*([^\n]+)"
    hubs_match = re.search(hubs_pattern, text)
    if hubs_match:
        metadata["hubs"] = [hub.strip() for hub in hubs_match.group(1).split(',')]
    
    return metadata

In [6]:
def parse_pdf_content(text, filename):
    """Анализирует содержимое PDF и извлекает нужные данные"""
    # Извлекаем метаданные
    metadata = extract_metadata(text)
    
    # Основной контент (исключаем первые строки, которые могут быть метаданными)
    content_lines = text.split('\n')[10:]  # Пропускаем первые 10 строк
    content = "\n".join([line for line in content_lines if line.strip()])
    
    return {
        "file_name": filename,
        "title": metadata["title"],
        "author": metadata["author"],
        "tags": ", ".join(metadata["tags"]),
        "hubs": ", ".join(metadata["hubs"]),
        "content": content,
        "char_count": len(text),
        "word_count": len(text.split())
    }

In [7]:
def extract_text_from_pdf(file_path):
    """Извлекает текст из PDF файла и возвращает объединенный текст"""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [8]:
def process_pdf_files(folder_path):
    """Обрабатывает все PDF файлы в указанной папке"""
    # Находим все PDF файлы
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print("PDF файлы не найдены в указанной папке")
        return None
    
    print(f"Найдено PDF файлов: {len(pdf_files)}")
    print("Список PDF файлов:")
    for i, pdf in enumerate(pdf_files, 1):
        print(f"{i}. {pdf}")
    
    # Парсим каждый PDF файл
    data = []
    for pdf_file in tqdm(pdf_files, desc="Обработка PDF файлов"):
        file_path = os.path.join(folder_path, pdf_file)
        text = extract_text_from_pdf(file_path)
        parsed_data = parse_pdf_content(text, pdf_file)
        data.append(parsed_data)
    
    return data

In [9]:
def save_to_csv(data, output_file):
    """Сохраняет данные в CSV файл"""
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Данные сохранены в {output_file}")

In [10]:
pdf_folder = "PDF_xabr"  # путь

# Обрабатываем PDF файлы
parsed_data = process_pdf_files(pdf_folder)

if parsed_data:
    # Создаем DataFrame
    df = pd.DataFrame(parsed_data)
    print("\nПример данных:")
    print(df.head())
    
    # Сохраняем в CSV
    output_csv = "parsed_pdf_data.csv"
    save_to_csv(parsed_data, output_csv)

Найдено PDF файлов: 26
Список PDF файлов:
1. Cocoapods, Carthage, SPM как выбрать менеджер зависимостей в iOS.pdf
2. Deutsche Telekom и Perplexity объявили о новом «AI Phone» стоимостью менее 1 000 долларов _ Хабр.pdf
3. Disrupt по делу как внедрять AI-продукты без розовых очков.pdf
4. imgonline-com-ua-site2pdfB28eKGLy6EjA.pdf
5. OSINT & Hacking — как работает фишинг для нельзяграма _ Хабр3.pdf
6. RAG борьба с низким качеством ответов в условия экономии памяти на GPU.pdf
7. Spring Boot 3.3 и Docker изменение в эффективных docker-образах.pdf
8. Yandex N.V. запретили в течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр.pdf
9. Быстрое начало работы с Gitlab CICD.pdf
10. Генеральный директор Mozilla покинула свой пост _ Хабр.pdf
11. За что безопасники будут гореть в аду_ _ Хабр.pdf
12. Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр.pdf
13. ИИ-агенты в Альфа-Банке_ нейросети создают автотесты без участия человека _ Хабр.pdf
14. Инструменты наблюдаемости, о которых нужн

Обработка PDF файлов:   0%|          | 0/26 [00:00<?, ?it/s]

Обработка PDF файлов: 100%|██████████| 26/26 [00:04<00:00,  5.87it/s]


Пример данных:
                                           file_name  \
0  Cocoapods, Carthage, SPM как выбрать менеджер ...   
1  Deutsche Telekom и Perplexity объявили о новом...   
2  Disrupt по делу как внедрять AI-продукты без р...   
3          imgonline-com-ua-site2pdfB28eKGLy6EjA.pdf   
4  OSINT & Hacking — как работает фишинг для нель...   

                                               title author  \
0                                             117.94          
1  Еще до начала MWC в Барселоне было очевидно, ч...          
2                                                 26          
3                                              71.07          
4                                              71.07          

                                                tags  \
0                                      red_mad_robot   
1  смартфоны, ии, искусственный интеллект, мобиль...   
2  ии, модели, нейросети, LLM, ai, продакшен, диз...   
3                              взлом, соцсет

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   26 non-null     object
 1   title       26 non-null     object
 2   author      26 non-null     object
 3   tags        26 non-null     object
 4   hubs        26 non-null     object
 5   content     26 non-null     object
 6   char_count  26 non-null     int64 
 7   word_count  26 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.8+ KB


In [12]:
df.head()

,file_name,title,author,tags,hubs,content,char_count,word_count
0,"Cocoapods, Carthage, SPM как выбрать менеджер ...",117.94,,red_mad_robot,Блог компании red_mad_robot,"Блог компании red_mad_robot\n, \nРазработка по...",27427,3624
1,Deutsche Telekom и Perplexity объявили о новом...,"Еще до начала MWC в Барселоне было очевидно, ч...",,"смартфоны, ии, искусственный интеллект, мобиль...","Искусственный интеллект, Смартфоны",стоимостью менее 1 000 долларов\n4 мин\n419\nИ...,6240,859
2,Disrupt по делу как внедрять AI-продукты без р...,26,,"ии, модели, нейросети, LLM, ai, продакшен, диз...","Управление проектами, Управление разработкой, ...",Как я стал core-разработчиком Python в 19 лет\...,22138,3075
3,imgonline-com-ua-site2pdfB28eKGLy6EjA.pdf,71.07,,"взлом, соцсети, osint","Блог компании Cloud4Y, Информационная безопасн...","3 мин\n 1.1K\nБлог компании Cloud4Y , Информа...",4979,704
4,OSINT & Hacking — как работает фишинг для нель...,71.07,,взлом,Блог компании Cloud4Y,Cloud4Y\nOSINT & Hacking — как работает фишинг...,4964,698
